In [1]:
import requests
from bs4 import BeautifulSoup
import openai
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from collections import Counter
import string
import re
from dotenv import load_dotenv
import os

load_dotenv()  # Load environment variables from .env file
openai.api_key = os.getenv("OPENAI_API_KEY")  # Load the API key from the environment variable

In [2]:
# Get the Wikipedia URL from the user
url = "https://en.wikipedia.org/wiki/Sun_Myung_Moon"

# Send a request to the URL and get the response
response = requests.get(url)

In [3]:
if response.status_code != 200:
    print("Failed")
    exit()

In [4]:
# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.content, "html.parser")

# Find the main content div and extract all paragraphs
content = soup.find("div", {"id": "mw-content-text"})
paragraphs = content.find_all("p")

# Concatenate all paragraph texts
article_text = ""
for p in paragraphs:
    article_text += p.text

# Print the article text
print("\nArticle text:\n")
print(article_text)


Article text:


Sun Myung Moon (Korean: 문선명; Hanja: 文鮮明; born Yong Myung Moon; 6 January 1920 – 3 September 2012) was a Korean religious leader, also known for his business ventures and support for conservative political causes.[1][2] A messiah claimant, he was the founder of the Unification movement (members of which consider him and his wife Hak Ja Han to be their "True Parents"),[3] and of its widely noted "Blessing" or mass wedding ceremonies, and the author of its unique theology the Divine Principle.[4][5][6] He was an anti-communist and an advocate for Korean reunification, for which he was recognized by the governments of both North and South Korea.[7] Businesses he promoted included News World Communications, an international news media corporation known for its American subsidiary The Washington Times,[8][9][10] and Tongil Group, a South Korean business group (chaebol),[11][12][13] as well as other related organizations.[1][14]
Moon was born in what is now North Korea. When 

In [5]:
prompt = f"You are a video script generator for a channel called Brief Bits. Please create a script for the following Wikipedia article:\n\n{article_text}"
chat_log = [{"role": "user", "content": prompt}]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",  # Use the desired engine
    messages=chat_log
)

summary = response.choices[0].message.content

chat_log.append({"role": "assistant", "content": summary})

# Print the summary
print("\nSummary:\n")
print(summary)


InvalidRequestError: This model's maximum context length is 4097 tokens. However, your messages resulted in 7545 tokens. Please reduce the length of the messages.

In [100]:
nltk.download('punkt')
nltk.download('stopwords')

def get_keywords(text, num_keywords=3):
    words = word_tokenize(text)
    words = [word.lower() for word in words if word.isalpha()]

    stop_words = set(stopwords.words("english"))
    filtered_words = [word for word in words if word not in stop_words]

    word_freq = Counter(filtered_words)

    keywords = [keyword for keyword, freq in word_freq.most_common(num_keywords)]

    return keywords


paragraphs = re.split(r'\n\s*\n', summary.strip())
keywords_per_paragraph = []

for paragraph in paragraphs:
    if len(paragraph.strip()) > 0:
        keywords = get_keywords(paragraph, num_keywords=3)
        keywords_per_paragraph.append(keywords)

print(keywords_per_paragraph)


[['welcome', 'brief', 'bits'], ['ice', 'national', 'oceanic'], ['pacific', 'ocean', 'source'], ['sound', 'noaa', 'vents'], ['ice', 'sounds', 'generated'], ['made', 'bloop', 'despite'], ['bloop', 'author', 'philip']]


[nltk_data] Downloading package punkt to /home/ash/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/ash/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [101]:
# Replace YOUR_API_KEY with your Bing Image Search API key
api_key = "0a9a33577e0446d7a76879cb0c70d7a3"
search_url = "https://api.bing.microsoft.com/v7.0/images/search"

# Set your keywords here

count = 10  # Number of images to retrieve

headers = {"Ocp-Apim-Subscription-Key": api_key}

paragraph_num = 1
for keywords in keywords_per_paragraph:
    params = {"q": keywords, "count": count, "imageType": "photo"}

    response = requests.get(search_url, headers=headers, params=params)
    response.raise_for_status()
    search_results = response.json()

    image_urls = [img["contentUrl"] for img in search_results["value"]]

    for i, image_url in enumerate(image_urls, 1):
        response = requests.get(image_url, stream=True)

        file_extension = os.path.splitext(image_url)[1]
        file_name = f"images/image_{paragraph_num}-{i}{file_extension}"
        paragraph_num += 1

        with open(file_name, "wb") as f:
            for chunk in response.iter_content(chunk_size=8192):
                if chunk:
                    f.write(chunk)
                    f.flush()

        print(f"Downloaded Image {i}: {file_name}")

Downloaded Image 1: images/image_1-1.jpg
Downloaded Image 2: images/image_2-2.jpg?v=1558351553
Downloaded Image 3: images/image_3-3.jpg
Downloaded Image 4: images/image_4-4.jpg?v=1565706398
Downloaded Image 5: images/image_5-5.jpg
Downloaded Image 6: images/image_6-6.jpg
Downloaded Image 7: images/image_7-7.jpg
Downloaded Image 8: images/image_8-8.jpg
Downloaded Image 9: images/image_9-9.jpg
Downloaded Image 10: images/image_10-10.png
Downloaded Image 1: images/image_11-1.jpg?width=982&height=726
Downloaded Image 2: images/image_12-2.jpg
Downloaded Image 3: images/image_13-3.jpg?fit=1600%2C1000&ssl=1
Downloaded Image 4: images/image_14-4.jpg
Downloaded Image 5: images/image_15-5.jpg
Downloaded Image 6: images/image_16-6.jpg
Downloaded Image 7: images/image_17-7.jpg
Downloaded Image 8: images/image_18-8.jpg
Downloaded Image 9: images/image_19-9.jpg?itok=zrYMHVk7
Downloaded Image 10: images/image_20-10.jpg
Downloaded Image 1: images/image_21-1.jpg
Downloaded Image 2: images/image_22-2.jp

In [102]:
url = "https://api.elevenlabs.io/v1/text-to-speech/EXAVITQu4vr4xnSDxMaL"
headers = {
    "accept": "audio/mpeg",
    "xi-api-key": "f41490cd2fb7d9caa62812426ce3d967",
    "Content-Type": "application/json",
}
data = {
    "text": summary,
    "voice_settings": {"stability": 0.75, "similarity_boost": 0.75},
}

response = requests.post(url, json=data, headers=headers)

if response.status_code == 200:
    with open("output.mp3", "wb") as f:
        f.write(response.content)
        print("Audio file saved as 'output.mp3'")
else:
    print(f"Error: {response.status_code} - {response.text}")

Audio file saved as 'output.mp3'
